# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps


# Import API key
from api_keys import weather_api_key,g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv("cities.csv")
weather_df.head()



SyntaxError: invalid syntax (<ipython-input-11-dd9725bc5ecb>, line 2)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["lat","lng"]]

# Filla NaN values
#locations = locations.dropna()
locations.head()


,lat,lng
0,-42.78,147.06
1,-54.81,-68.31
2,62.45,-114.38
3,11.00,122.67
4,-0.74,-90.35


In [4]:
# Defining Humidity as weight
weights = weather_df["humidity"].astype(float)

In [5]:
#Set up your map
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow new DF to the 3 criterias defined for the exercise
weather2 = weather_df
weather2 = weather2[(weather2["max_temp"] > 70)]
weather2 = weather2[(weather2["humidity"] < 80)] 
weather2 = weather2[(weather2["windspeed"] < 10)]
weather2 = weather2[(weather2["cloudiness"] == 0)]

# drop null
weather2 = weather2.dropna()
weather2.count()

city          7
lat           7
lng           7
max_temp      7
humidity      7
cloudiness    7
windspeed     7
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df.
hotel_df = weather2

# set up additional column to hold information
hotel_df['Hotel Name'] = " "
hotel_df.head()

,city,lat,lng,max_temp,humidity,cloudiness,windspeed,Hotel Name
39,Jamestown,-33.21,138.60,74.52,41,0,8.01,
171,Moyale,3.52,39.05,75.40,55,0,9.28,
311,La Rioja,-29.41,-66.86,70.57,34,0,3.91,
353,Poli,8.48,13.24,71.80,27,0,4.79,
371,Guerrero Negro,27.97,-114.04,72.43,28,0,3.98,


In [8]:
# find the closest hotel of each type to coordinates
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # convert to json
    data = response.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
hotel_df


,city,lat,lng,max_temp,humidity,cloudiness,windspeed,Hotel Name
39,Jamestown,-33.21,138.60,74.52,41,0,8.01,
171,Moyale,3.52,39.05,75.40,55,0,9.28,
311,La Rioja,-29.41,-66.86,70.57,34,0,3.91,
353,Poli,8.48,13.24,71.80,27,0,4.79,
371,Guerrero Negro,27.97,-114.04,72.43,28,0,3.98,
407,Rio Cuarto,-33.13,-64.35,73.40,40,0,3.36,
478,Tabou,4.42,-7.36,71.53,70,0,2.80,


In [10]:
fig = gmaps.figure(layout={
        'padding': '3px',
        'border': '1px solid black'
})
locations2 = hotel_df[["lat", "lng"]]
fig.add_layer(gmaps.heatmap_layer(locations2))
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt>
<dd>Hotel Name</dd>
<dt>city</dt>
<dd>City</dd>
<dt>Country</dt>
<dd>Country</dd>
</dl>

hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
 #Display Map
fig

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-10-d890976b392e>, line 20)

In [ ]:

#<dd>{Hotel Name}</dd>
#<dt>city</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>